In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/toxigen_explanations_v2/toxigen_explanations.csv')
len(df)

32582

In [20]:
def get_offensiveness_binary(data):
    data.fillna(' ', inplace = True)
    off_data = data['offensiveness']
    offensive = []
    for i, row in enumerate(off_data.str.split(',', expand=False)):
        #if not isinstance(row, float):
        offensive.append(row[0])
            
    offensive = list(map(lambda x: x.lstrip().lower(), offensive))
    
    offensive = list(map(lambda x: x.replace('very ', ''), offensive))
    offensive = list(map(lambda x: x.replace('highly ', ''), offensive))
    offensive = list(map(lambda x: x.replace('extremely ', ''), offensive))
    
    for i, off in enumerate(offensive):
        if 'anti-sem' in off:
            offensive[i] = 'anti-semitic'
        if 'none' in off or 'not offensive' in off or \
             'not necessarily offensive' in off or 'n/a' in off or \
             'not applicable' in off:
            offensive[i] = 'not offensive'
        if 'ableis' in off:
            offensive[i] = 'ableist'
        if 'racis' in off:
            offensive[i] = 'racist'
        if 'stereotype' in off:
            offensive[i] = 'offensive stereotype'

    off_binary = (np.array(offensive) != 'not offensive').astype(int)
    return off_binary
    

In [21]:
off_binary = get_offensiveness_binary(df)
df['offensive_binary'] = off_binary

In [22]:
df['offensive_binary'].value_counts()

1    29032
0     3550
Name: offensive_binary, dtype: int64

In [28]:
df_nontoxic = df[df['offensive_binary']==0]
non_toxic_statements = set(df_nontoxic['statement'].values)
df_nontoxic_related = df.groupby("statement").filter(
            lambda x: x["statement"].values[0] in non_toxic_statements
        )

In [30]:
len(df_nontoxic_related)


5678

In [31]:
df_nontoxic_related.to_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/toxigen_explanations_v2/toxigen_explanations_nontoxic.csv', index=False)